In [ ]:
# | default_exp solvemodels

## Setup

In [ ]:
# | export


import json
import sys

import cobra
# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.buildingediting
import mmon_gcm.solving
import mmon_gcm.supermodel
import pandas as pd

In [ ]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is where to adjust whether fva is run if running in notebook
args = {
    "run_fva": "True",
    "no_processes": 3,
    "output_dir": "../outputs/model_solutions/",
    "model_file": "../models/4_stage_GC.xml",
    "map_file": "../inputs/map.json",
    "parameters_file": "../inputs/arabidopsis_parameters.csv",

}

sys.argv = ["script_name"] + list(args.values())

In [ ]:
# | export

run_fva = sys.argv[1]
if run_fva == "True":
    run_fva = True
elif run_fva == "False":
    run_fva = False
else:
    raise ValueError(f"Please specify True or False for run_fva, not {run_fva}")

no_processes = int(sys.argv[2])
output_dir = sys.argv[3]
model_file = sys.argv[4]
map_file = sys.argv[5]
parameters_file = sys.argv[6]

## Instantiate and apply base constraints to model

### Import FBA model

In [ ]:
# | export

four_stage_GC_model = cobra.io.sbml.read_sbml_model(model_file)  # read model

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


No objective coefficients in model. Unclear what should be optimized


Decrease tolerance

In [ ]:
# | export

print(four_stage_GC_model.solver.configuration.tolerances.integrality)
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)
four_stage_GC_model.solver.configuration.tolerances.feasibility = 1e-8  # 1e-9 takes a long time to solve
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)

1e-07
1e-07
1e-08


### Define reactions to use for FVA

In [ ]:
# | export

if run_fva == True:

    linker_list = [
        reaction
        for reaction in four_stage_GC_model.reactions
        if "gc_Linker_2" in reaction.id
        or "gc_Linker_1" in reaction.id
        or "_total_pseudolinker_1" in reaction.id
        or "_total_pseudolinker_2" in reaction.id
        or "ae_gc" in reaction.id
    ]

    extra_fva_rxns = set()

    with open(map_file, "r+") as f:
        map_data = json.load(f)

    for reaction in map_data[1]["reactions"].values():
        extra_fva_rxns.add(four_stage_GC_model.reactions.get_by_id(reaction["bigg_id"]))

    fva_list = list(set(linker_list) | extra_fva_rxns)

else:
    fva_list = []

### Instantiate Supermodel

Import parameters from csv, created in [parameters notebook](02_parameters.ipynb)

In [ ]:
# | export

parameters_df = pd.read_csv(parameters_file, index_col=0)
parameters_df

,Value,Units,Source
P_abs,9.000000e-01,Dimensionless,"Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,m,Wuyts et al. (2010)
A_l,1.000000e+00,m$^2$,Fixed
V_gc_ind,4.750000e-13,dm$^3$,Jezek and Blatt (2017)
FqFm,9.000000e-01,Dimensionless,Lawson (2003)
R_ch,6.923077e-02,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)"
L_air,3.700000e-01,Dimensionless,Earles et al. (2018)
L_epidermis,1.500000e-01,Dimensionless,Wuyts et al. (2010)
Vac_frac,7.510000e-01,Dimensionless,Wang et al. (2017)
T,2.961500e+02,K,Horrer et al. (2016)


In [ ]:
# | export

arabidopsis_supermodel = mmon_gcm.supermodel.SuperModel(parameters_df.loc[:, "Value"], fba_model=four_stage_GC_model)

/home/nls119/google_drive/plant_cell/mmon-gcm/mmon_gcm/supermodel.py:25: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for parameter, value in parameters.iteritems():


In [ ]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True);

Volume in phase 0 is 0.38pL
Volume in phase 1 is 0.4375pL
Volume in phase 2 is 0.38pL
Volume in phase 3 is 0.38pL


This is a little different to Jezek and Blatt (2017), where they have closed as 0.3-0.4pL and open as 0.5-0.65. Open apertures are on the small side but still within their range (2-6). Wang used a volume of 0.4pL.

### Constrain SuperModel

#### Constrain osmolarity of the model using the equation from the OnGuard model (Hills et al 2012)

In [ ]:
# | export

arabidopsis_supermodel.constrain_osmolarity(printouts=True);

Raw osmolarities: [0.039359327616195004, 0.05492236409486689, 0.039359327616195004, 0.039359327616195004]
Osmolarities in mM: [178.58134127 216.44281417 178.58134127 178.58134127]
Change in osmolarity: 37.86147289566091mM
c osmolarities constrained to [0.00980047 0.01367567 0.00980047 0.00980047]
v osmolarities constrained to [0.02955886 0.0412467  0.02955886 0.02955886]


In [ ]:
import numpy as np

In [ ]:
np.array(arabidopsis_supermodel.get_osmolarities())

array([0.03935933, 0.05492236, 0.03935933, 0.03935933])

In [ ]:
help(arabidopsis_supermodel.get_osmolarities)

Help on method get_osmolarities in module mmon_gcm.supermodel:

get_osmolarities(apertures='default', equation='onguard', printouts=False) method of mmon_gcm.supermodel.SuperModel instance
    Calculates the osmolarities at each phase of the model using the equation
    specified, default is onguard. Adds these as an attribute to the SuperModel



In [ ]:
mmoles_per_m2 = np.array(arabidopsis_supermodel.get_osmolarities())
mmoles_per_gc = mmoles_per_m2 / arabidopsis_supermodel.N_gcs
moles_per_gc = mmoles_per_gc * 10 ** -3
fmoles_per_gc = moles_per_gc / 10 ** -15
print(f"Closed fmoles: {fmoles_per_gc[0]}")
print(f"Open fmoles: {fmoles_per_gc[1]}")
print(f"Increase in fmoles: {fmoles_per_gc[1] - fmoles_per_gc[0]}")

Closed fmoles: 67.86090968309483
Open fmoles: 94.69373119804636
Increase in fmoles: 26.832821514951533


#### Constrain photons using a PPFD of 150µmolm$^{-2}$s$^{-1}$, same as used in Horrer et al (2016)

In [ ]:
# | export

arabidopsis_supermodel.constrain_photons(150, printouts=True);

Total leaf volume: 0.17dm3
Guard cell volume in 1m2 leaf: 0.0003dm3
Mesophyll cell volume in 1m2 leaf: 0.091dm3
Proportion of the leaf that is gc: 0.003
PPFD 150umolphotonsm-2
Photon influx into leaf: 486.0 mmolphotonsm-2hr-1
e = 0.062
Photon influx into Guard cells: 0.092mmolphotonsm-2hr-1
Photon influx into mesophyll cells: 485.908mmolphotonsm-2hr-1


In [ ]:
# | export

arabidopsis_supermodel.add_maintenance();

## Blue Light

### ATPase unconstrained

#### WT

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0

    blue_unconstrained_wt, blue_unconstrained_wt_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

Running pFBA
Running FVA on 346 reactions using 3 processes


/home/nls119/lib/miniconda3/envs/mmon-gcm/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/nls119/lib/miniconda3/envs/mmon-gcm/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [ ]:
# | export

blue_unconstrained_wt_solution.to_csv(output_dir + "blue_unconstrained_wt.csv")

#### Starch KO

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    blue_unconstrained_starchko, blue_unconstrained_starchko_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

Running pFBA
Running FVA on 346 reactions using 3 processes


In [ ]:
# | export

blue_unconstrained_starchko_solution.to_csv(output_dir + "blue_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    blue_constrained_wt, blue_constrained_wt_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

Running pFBA
Running FVA on 346 reactions using 3 processes


In [ ]:
# | export

blue_constrained_wt_solution.to_csv(output_dir + "blue_constrained_wt.csv")

#### Starch KO

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    blue_constrained_starchko, blue_constrained_starchko_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

blue_constrained_starchko_solution.to_csv(output_dir + "blue_constrained_starchko.csv")

## White Light

### Unconstrained ATPase

#### WT

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    white_unconstrained_wt, white_unconstrained_wt_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

white_unconstrained_wt_solution.to_csv(output_dir + "white_unconstrained_wt.csv")

#### Starch KO

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    white_unconstrained_starchko, white_unconstrained_starchko_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

white_unconstrained_starchko_solution.to_csv(output_dir + "white_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    white_constrained_wt, white_constrained_wt_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

white_constrained_wt_solution.to_csv(output_dir + "white_constrained_wt.csv")

#### Starch KO

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    white_constrained_starchko, white_constrained_starchko_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

white_constrained_starchko_solution.to_csv(output_dir + "white_constrained_starchko.csv")

## Guard cell has no photosynthetic capacity

### Unconstrained ATPase

#### WT

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    nops_unconstrained_wt, nops_unconstrained_wt_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

nops_unconstrained_wt_solution.to_csv(output_dir + "nops_unconstrained_wt.csv")

#### Starch KO

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    nops_unconstrained_starchko, nops_unconstrained_starchko_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

In [ ]:
# | export

nops_unconstrained_starchko_solution.to_csv(output_dir + "nops_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    nops_unconstrained_wt, nops_unconstrained_wt_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

Running pFBA
Running FVA on 346 reactions using 3 processes


In [ ]:
# | export

nops_unconstrained_wt_solution.to_csv(output_dir + "nops_constrained_wt.csv")

#### Starch KO

In [ ]:
# | export

with arabidopsis_supermodel.fba_model as m:

    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    nops_unconstrained_starchko, nops_unconstrained_starchko_solution = mmon_gcm.solving.get_pfba_fva_solution(
        m, rxn_list=fva_list, processes=no_processes)

Running pFBA
Running FVA on 346 reactions using 3 processes


In [ ]:
# | export

nops_unconstrained_starchko_solution.to_csv(output_dir + "nops_constrained_starchko.csv")